In [ ]:
[1]*10**10 #get more ram quickly

In [1]:
from google.colab import drive
 
drive.mount('/gdrive')
# the project's folder
%cd /gdrive/'My Drive'/CDIP Dataset

Mounted at /gdrive
/gdrive/My Drive/CDIP Dataset


In [ ]:
#I unzip images folder
from zipfile import ZipFile
filename='test_data.zip'
with ZipFile(filename,'r') as zip:
  zip.extractall()
  print("done")

done


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import os
import codecs

from nltk.corpus import stopwords

from keras.utils import to_categorical
import numpy as np
from sklearn.metrics import accuracy_score, precision_score,f1_score, recall_score, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.random import seed
seed(1)
from keras.models import Model, Sequential
from keras.layers import Embedding, Dense, Input, Conv1D, MaxPool1D, Concatenate, Flatten, Dropout
from keras.layers import ZeroPadding1D, Activation, GlobalMaxPool1D
from keras.regularizers import l2


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from numpy import asarray


In [5]:
def load_fast_test_embeddings(max_words, best_feat_list, tokenizer_index):
    f = open('/gdrive/My Drive/CDIP Dataset/cvl/Fast_text-emb/wiki-news-300d-1M.vec')
    embeddings_index = {}
    comp=0
    for line in f:
      values = line.split()
      word = values[0]
      coefs =asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
    embedding_dim = 300 # fastext dim 
    embedding_matrix = np.zeros((max_words, embedding_dim))
    for word, i in tokenizer_index.items():
      if i < max_words and word in best_feat_list:
        comp +=1
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector
    print("comp",comp)
    return embedding_dim, embedding_matrix


In [13]:
ensemble_labels = ['Adver','email','handwriten','letter']

In [14]:
#for training data
texts_train = []
labels_train = []
not_read=[]
include_extension =['txt']
#train_dir = os.path.join(Dataset_Dir,'train')
for label_index, label_name in enumerate(ensemble_labels):
  base = '/gdrive/My Drive/CDIP Dataset/cvl/train_data/'+label_name
  print(base)
  #text files selection
  files = [fn for fn in os.listdir(base) if any(fn.endswith(ext) for ext in include_extension)]
  for  file in files:
    #print(file)
    fr = codecs.open(base+'/'+file,encoding='utf-8',errors='ignore')
    #print("ok1")

    #text =fr.read()
    #if not fr.read():
    #  not_read.append(file)
    #  continue
    text =fr.read().replace('\n','')

    #print("ok2")
    #print(fr.read())
    texts_train.append(text)
    labels_train.append(label_index)
    fr.close()
      
           
print('Length of Training Text: %s' % len(texts_train))
print('Length of  Training Labels: %s' % len(labels_train))

/gdrive/My Drive/CDIP Dataset/cvl/train_data/Adver
/gdrive/My Drive/CDIP Dataset/cvl/train_data/email
/gdrive/My Drive/CDIP Dataset/cvl/train_data/handwriten
/gdrive/My Drive/CDIP Dataset/cvl/train_data/letter
Length of Training Text: 1310
Length of  Training Labels: 1310


In [15]:
texts_train[10]

'last hunt, major hooumsmoked cigarotte stamped withhis family crest,now everybody stamped ownfamily crest, .walmost then, try tobe.)'

In [16]:
#for training data
texts_test = []
labels_test = []
include_extension =['txt']

for label_index, label_name in enumerate(ensemble_labels):
  base = '/gdrive/My Drive/CDIP Dataset/cvl/test_data/'+label_name
  #text files selection
  files = [fn for fn in os.listdir(base) if any(fn.endswith(ext) for ext in include_extension)]
  for  file in files:
    fr = codecs.open(base+'/'+file,encoding='utf-8',errors='ignore')
    texts_test.append(fr.read().replace('\n', ''))
    labels_test.append(label_index)
    fr.close()
      
           
print('Length of Test Text: %s' % len(texts_test))
print('Length of  Test Labels: %s' % len(labels_test))

Length of Test Text: 189
Length of  Test Labels: 189


In [17]:
texts_test[10]

"the new lookingold# and general's warnin' jokin:causes lung cancer, hear ase,emphy and may complicate empane)"

In [18]:
corpus = texts_train + texts_test

In [10]:
# select n features from each class
number_of_features_each_class=500
best_feat_list=list()
path_results_features_selection = '/gdrive/My Drive/CDIP Dataset/cvl/best_features/'
ensemble_labels = ['Adver','email','handwriten','letter']
for label in ensemble_labels:
  current_label=path_results_features_selection+label
  filename=label+'.txt'
  fr = codecs.open(current_label+'/'+filename,encoding='utf-8',errors='ignore')
  best_features=fr.read().split()
  for feature in range(0, number_of_features_each_class):
    if feature > len(best_features) - 1:
      break
    if best_features[feature] not in best_feat_list:
      best_feat_list.append(best_features[feature])
  #select feature for label




   


In [13]:
best_feat_list

['would',
 'please',
 'surgeon',
 "general's",
 'warning:',
 'know',
 'and',
 'york,',
 'dr.',
 'low',
 'meeting',
 'your',
 'has',
 'also',
 'modified',
 'information',
 'original',
 'smoking',
 'new',
 'work',
 'tobacco',
 'cigarette',
 'product',
 'research',
 'copy',
 'lung',
 'york',
 'let',
 'forward',
 'need',
 'see',
 'taste',
 'october',
 'time',
 'following',
 'january',
 'two',
 'women',
 'june',
 'next',
 'dangerous',
 'park',
 'you',
 'august',
 'pregnant',
 'september',
 'hope',
 'this',
 'provide',
 'call',
 'july',
 'february',
 'received',
 'tar',
 'dear',
 'use',
 'find',
 'march',
 'like',
 'heart',
 'april',
 'make',
 'general',
 'could',
 'flavor',
 'contains',
 'list',
 'number',
 'office',
 'end',
 'november',
 'smoke',
 'well',
 'give',
 'birth',
 'complicate',
 'premature',
 'might',
 'eee',
 'box',
 'additional',
 'interest',
 'serious',
 'week',
 'last',
 'current',
 'recent',
 'within',
 'carbon',
 'december',
 'reduces',
 'program',
 'year',
 'east',
 'birt

In [19]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
maxlen = 200  # length
max_words = 10000  # most common words
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(corpus)
tokenizer_index = tokenizer.word_index
print('Found %s unique tokens.' % len(tokenizer_index))

texts_train = asarray(tokenizer.texts_to_sequences(texts_train))
texts_test = asarray(tokenizer.texts_to_sequences(texts_test))

texts_train = pad_sequences(texts_train, maxlen=maxlen)
texts_test = pad_sequences(texts_test, maxlen=maxlen)

labels_train = asarray(labels_train)

labels_test = asarray(labels_test)

Found 35511 unique tokens.


In [21]:
import pickle

# saving
with open('/gdrive/My Drive/CDIP Dataset/cvl/Vocab/tokenizer_voc.pickle', 'wb') as tokenizer_vocab:
  pickle.dump(tokenizer, tokenizer_vocab, protocol=pickle.HIGHEST_PROTOCOL)


# loading
#with open('/gdrive/My Drive/Text_model_series/tokenizer_voc.pickle', 'rb') as tokenizer_vocab:
  #  tokenizer = pickle.load(tokenizer_vocab)

In [ ]:
labels_train

array([0, 0, 0, ..., 4, 4, 4])

In [15]:
X_train, y_train = texts_train, labels_train
X_test, y_test = texts_test, labels_test

print('Shape of X_train tensor:', X_train.shape)
print('Shape of y_train tensor:', y_train.shape)
print('Shape of X_test tensor:', X_test.shape)
print('Shape of y_test tensor:', y_test.shape)

Shape of X_train tensor: (1310, 200)
Shape of y_train tensor: (1310,)
Shape of X_test tensor: (189, 200)
Shape of y_test tensor: (189,)


In [28]:
embedding_dim,embedding_matrix=load_fast_test_embeddings(max_words, best_feat_list, tokenizer_index)
text_seq_input = Input(shape=(maxlen,), dtype='int32')
text_embedding1 = Embedding(max_words, embedding_dim, input_length=maxlen,weights=[embedding_matrix], trainable=True)(text_seq_input)
text_embedding2 = Embedding(max_words, embedding_dim, input_length=maxlen)(text_seq_input)
text_embedding3 = Embedding(max_words, embedding_dim, input_length=maxlen)(text_seq_input)

filter_sizes = [3, 5]
conv_pools = []
for text_embedding in [text_embedding1, text_embedding2, text_embedding3]:
  for filter_size in filter_sizes:
    l_zero = ZeroPadding1D((filter_size - 1, filter_size - 1))(text_embedding)
    l_conv = Conv1D(filters=16, kernel_size=filter_size, padding='same', activation='tanh')(l_zero)
    l_pool = GlobalMaxPool1D()(l_conv)
    conv_pools.append(l_pool)

l_merge = Concatenate(axis=1)(conv_pools)
l_dense = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(l_merge)
l_out = Dense(4, activation='softmax')(l_dense)
model = Model(inputs=[text_seq_input], outputs=l_out)
model.summary()



Found 1000000 word vectors.
comp 1008
Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 225)]        0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 225, 300)     3000000     input_4[0][0]                    
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 225, 300)     3000000     input_4[0][0]                    
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 225, 300)     3000000     input_4[0][0]                    
_________________________________________________

In [17]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [18]:

model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train,epochs=20,batch_size=50,validation_data=(X_test, y_test),verbose=1)


Epoch 1/20
27/27 [==============================] - 2s 65ms/step - loss: 1.9434 - accuracy: 0.5802 - val_loss: 1.4845 - val_accuracy: 0.7302
Epoch 2/20
27/27 [==============================] - 1s 38ms/step - loss: 1.1651 - accuracy: 0.8557 - val_loss: 1.0689 - val_accuracy: 0.8360
Epoch 3/20
27/27 [==============================] - 1s 38ms/step - loss: 0.7400 - accuracy: 0.9443 - val_loss: 0.8346 - val_accuracy: 0.8571
Epoch 4/20
27/27 [==============================] - 1s 38ms/step - loss: 0.4774 - accuracy: 0.9740 - val_loss: 0.6662 - val_accuracy: 0.8571
Epoch 5/20
27/27 [==============================] - 1s 37ms/step - loss: 0.3118 - accuracy: 0.9832 - val_loss: 0.5584 - val_accuracy: 0.8677
Epoch 6/20
27/27 [==============================] - 1s 38ms/step - loss: 0.2031 - accuracy: 0.9878 - val_loss: 0.6376 - val_accuracy: 0.7937
Epoch 7/20
27/27 [==============================] - 1s 37ms/step - loss: 0.1441 - accuracy: 0.9847 - val_loss: 0.4751 - val_accuracy: 0.8519
Epoch 8/20
27

In [19]:
model.save('/gdrive/My Drive/CDIP Dataset/cvl/text_model_cvl_cdip.hdf5')